## Imports

In [2]:
#!pip install undetected-chromedriver
import re
import time
import codecs
import os
import undetected_chromedriver as uc

## Function to scrape links of courses

In [2]:
def get_rank(webpage, outputfile):
    """
    Purpose: obtain COURSE list from the input web page.
    :param webpage: the webpage that has products.
    :param outputfile: output file that has the product information.
    """
    count = 1
    page = 1
    f = open(outputfile, 'wt')
    f.write('id\tname\tlink\tseller\tprice\toriginal_price\tlectures\thours\tlevel\t' +
            time.strftime("%c") + '\n')
    # driver = webdriver.Chrome()
    options = uc.ChromeOptions()
    driver = uc.Chrome()
    
    while count < 1000:
        if count == 1:
            tem_webpage = webpage
        else:
            page += 1
            tem_webpage = webpage + "?p=" + str(page)

        driver.get(tem_webpage)
        time.sleep(30)
    
        # parse the elements in the webpage.
        xpath_product = "//div[@class='popper--popper--2r2To']"
        product = driver.find_elements_by_xpath(xpath_product)
    
        for i in range(len(product)):
            try:
                link = product[i].find_element_by_class_name("browse-course-card--link--3KIkQ").get_attribute('href')#getting class where course links are present 
                name = link.replace('https://www.udemy.com/course/', '') 
                #time.sleep(100)
            except:
                continue

            try:
                tem_seller = product[i].find_element_by_class_name(
                    "course-card--instructor-list--nH1OC").text #Getting the seller name information
                seller = re.sub('[^A-Za-z0-9_]', ' ', tem_seller) #Removing special characters from names
                #time.sleep(100)
            except:
                seller = "NULL"

            try:
                tem_price = product[i].find_element_by_class_name(
                    "course-card--discount-price--1bQ5Q").text   #Getting current price of course
                price = tem_price.replace("$", "").replace("Current price", "").replace("\n", "") #Removing "$" and characters from price 
                #time.sleep(100)
            except:
                price = 'NULL'

            try:
                tem_original_price = product[i].find_element_by_class_name(
                    "price-text--original-price--1sDdx").text    #Getting the original price
                original_price = tem_original_price.replace("$", "").replace("Original Price", "").replace("\n","") #Getting only digits
                #time.sleep(100)
            except:
                original_price = 'NULL'

            try:
                other_info = product[i].find_element_by_class_name("course-card--course-meta-info--2jTzN").text #Getting no of lectures, total hours, level 
                other_info = other_info.replace("\n", '').replace("lectures", ',').replace("total hours", ',')
                other_info = other_info.replace("lecture", ',').replace("hour", ',')
                other_info = other_info.split(',')
                lectures = other_info[1]
                hours = other_info[0]
                level = other_info[2]
                #time.sleep(100)

            except:
                continue
                # other_info = 'NULL'

            f.write(str(count) + '\t' + name + '\t' + link + '\t' + seller + '\t' + price + '\t'
                    + original_price + '\t' + lectures + '\t' + hours + '\t' + level + '\n') #Saving the data to file

            count += 1

    driver.quit()
    f.close()


if __name__ == "__main__":
    webpage = "https://www.udemy.com/courses/development/data-science"
    outputfile = "course_list_all.tsv"
    get_rank(webpage, outputfile)

<ipython-input-2-2bc558789210>:32: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  product = driver.find_elements_by_xpath(xpath_product)


KeyboardInterrupt: 

## Used interrupt above because I got links for more than 2000 data science courses.

## Class and Functions to Scrape Udemy course data like Descriptions

In [ ]:
class get_course_data:
    """
    A class to get course data from course webpage.
    """
    def __init__(self,inputfile0,output_directory0):
        """
        Initialize class with input file and output directory.
        :param inputfile0: input file.
        :param output_directory0: output directory.
        """
        with open(inputfile0) as file:
            content = file.readlines()
        self.links = [item.split('\t')[2] for item in content[1:]]
        self.output_directory = output_directory0

        # self.driver = webdriver.Edge()
        # self.driver=webdriver.Chrome()
        # self.driver = webdriver.Chrome('/Users/15512/Downloads/chromedriver')
        options = uc.ChromeOptions()
        self.driver = uc.Chrome()
        

    def get_data(self,k):
        """
        Get data of the k-th course.
        :param k: course number.
        """
        product_link=self.links[k]
        self.driver.get(product_link) #Getting the kth course webpage
        time.sleep(30) #Giving time for the page to parse or load data. 30 seconds because Udemy was talking time to load webpage

        course_file_name0 = self.output_directory+str(k+1)+"_info.txt" #Storing all course data in individual txt files so even if I interrupt the code I dont loose data and I can start working with existing data. 

        try:
            title= self.driver.find_element_by_class_name("clp-lead__title").text #Getting the title
            title= re.sub('[^A-Za-z0-9_]', ' ', title) #Removing special characters 
        except:
            return
        try:
            headline = self.driver.find_element_by_class_name("clp-lead__headline").text #Getting the course headline
            headline= re.sub('\t\n\r', '', headline) 
        except:
            headline = "NULL"

        try:
            enrollment = self.driver.find_elements_by_xpath("//div[@data-purpose='enrollment']") #Getting total no of enrollments
            enrollment = re.sub(',', '', enrollment[1].text)
            enrollment =re.sub("[^0-9]", '', enrollment) #Removing anthing that is not a digit
        except:
            enrollment = "NULL"

        try:
            ratings_a = self.driver.find_element_by_xpath("//div[@class='styles--rating-wrapper--5a0Tr']").text #Getting the ratings 
            rating = ratings_a.split('\n')[1]
            num_ratings = ratings_a.split('\n')[2]
            num_ratings=re.sub("[^0-9]", '', num_ratings)
        except:
            num_ratings = "NULL"   
            rating = "NULL"     

        try:
            last_update_date = self.driver.find_element_by_xpath("//div[@data-purpose='last-update-date']").text #Getting the date on which the course was updated
            last_update_date= re.sub('Last updated ', '', last_update_date)
        except:
            last_update_date = "NULL"        

        try:
            what_you_get = self.driver.find_element_by_class_name("what-you-will-learn--content-spacing--3btHJ").text #Getting the course objectives
            what_you_get= re.sub('[^A-Za-z0-9_]', ' ', what_you_get)
        except:
            what_you_get = "NULL"

        try:
            requirements = self.driver.find_element_by_class_name("ud-component--course-landing-page-udlite--requirements").text #Getting prerequisites
            requirements = re.sub('[^A-Za-z0-9_]', ' ', requirements)
        except:
            requirements = "NULL"

        try:
            description = self.driver.find_element_by_xpath("//div[@data-purpose='course-description']").text #Getting course description
            description= re.sub('\n', ' ', description)
            description= re.sub('\t', ' ', description)
        except:
            description = "NULL"

        try:
            stars = self.driver.find_elements_by_class_name("review-summary-widget--rate-percent--2dtfO") #Getting the star ratings
            five_stars = stars[0].text
            four_stars = stars[1].text
            three_stars = stars[2].text
            two_stars = stars[3].text
            one_star = stars[4].text
        except:
            five_stars = "NULL"
            four_stars = "NULL"
            three_stars = "NULL"
            two_stars = "NULL"
            one_star = "NULL"

        f0 = codecs.open(course_file_name0, 'w', encoding='utf8')
        # Writing all the data to the file
        f0.write(str(k+1)+'\n')
        f0.write(title + '\n')
        f0.write(headline +'\n')
        f0.write(enrollment +'\n')
        f0.write(rating +'\n')
        f0.write(num_ratings +'\n')
        f0.write(last_update_date +'\n')
        f0.write(what_you_get +'\n')
        f0.write(requirements +'\n')
        f0.write(description +'\n')

        f0.write(five_stars +'\n')
        f0.write(four_stars +'\n')
        f0.write(three_stars +'\n')
        f0.write(two_stars +'\n')
        f0.write(one_star +'\n')

        f0.close()
        
    def run(self):
        """
        Run the web scrapping.
        """
        for k in range(0,len(self.links)):
            self.get_data(k)
        self.driver.quit()

if __name__ == "__main__":
    # Specify the file which has all links.
    file = "course_list_all.tsv"
    # Specify the output file.
    output_directory = 'C:/Users/15512/Desktop/Udemy_data/data'

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    course = get_course_data(file, output_directory)
    course.run()

## We just create an Object of above class and run code cells above to scrape Udemy data